In [1]:
import pandas as pd
import numpy as np
import polars as pl

In [2]:
df = pl.read_csv("data/3_stock_trading_hrs.csv")
# Ensure TIME_M is parsed as a datetime column with strict=False to handle invalid formats
df = df.with_columns(
    pl.col("TIME_M").str.to_time(format="%H:%M:%S%.f").alias("TIME_M")
)

# Filter rows between 9:30:00 and 16:00:00
df = df.filter(
    pl.col("TIME_M").is_between(pl.time(9, 30), pl.time(16, 0))
)

df.shape

(20925163, 14)

In [3]:
df['SYM_ROOT'].value_counts()

SYM_ROOT,count
str,u32
"""AAPL""",11667035
"""NVDA""",3526742
"""TSLA""",5731386


In [43]:
df['QU_CANCEL'].value_counts()

QU_CANCEL,count
str,u32
null,20925163


# TAQ Quote - Data Dictionary
Exchange that issued the quote (EX)

Bid price (BID)

Bid size in units of trade (BIDSIZ)

Ask price (ASK)

Ask size in units of trade (ASKSIZ)

Condition of quote issued (QU_COND)

Bid exchange (BIDEX)

Ask exchange (ASKEX)

Quote Sequence Number (QU_SEQNUM)

National BBO Indicator (NATBBO_IND)

NASD BBO Indicator (NASDBBO_IND)

Quote Cancel/Correction (QU_CANCEL)

Source of Quote (QU_SOURCE)

## Quote Condition
‘R’ = Regular, two-sided open quote

‘L’ = Closed Market Maker (NASD)

‘Y’ = Regular - One Sided Quote (NASDAQ)

# single exchange
1. find the most liquid exchange ==> narrow apread
    - filter all two-sided quotes and get the narrowest average for each exchange
2. filter for liquid exchange

In [44]:
df['QU_COND'].value_counts()

QU_COND,count
str,u32
"""Y""",15
"""R""",20925148


In [45]:
# Filter for rows where QU_COND is 'R'
filtered_df = df.filter(pl.col("QU_COND") == "R")

# Create a new column 'spread' calculated as ASK - BID
filtered_df = filtered_df.with_columns((pl.col("ASK") - pl.col("BID")).alias("spread"))

# Aggregate by EX column and calculate the average spread
average_spread = (
	filtered_df
	.group_by("EX")
	.agg(pl.col("spread").mean().alias("average_spread"),
        pl.col("spread").count().alias("count"))
    .sort("count")
)

average_spread

EX,average_spread,count
str,f64,u32
"""L""",0.0,2
"""A""",2.037869,7957
"""C""",2.054645,12553
"""H""",0.189494,531024
"""X""",0.423976,623698
…,…,…
"""P""",0.039858,1577094
"""Z""",0.042918,2436341
"""K""",0.035883,2549975


*** EX Q (NASDAQ) has the tightest spread, also has the highest refresh. We assume we can connect directly with NASDAQ ***

for 10 May aapl stock: 
- K (Direct X) performance is very bad
- Z (BATS) performance is still reasonable

In [46]:
filtered_df.filter(pl.col("EX").is_in(["Q"]))['spread'].describe()
# filtered_df.filter(pl.col("EX").is_in(["K"]))['spread'].describe()
# filtered_df.filter(pl.col("EX").is_in(["Z"]))['spread'].describe()

statistic,value
str,f64
"""count""",3.171159e6
"""null_count""",0.0
"""mean""",0.025867
"""std""",0.024503
"""min""",0.01
"""25%""",0.01
"""50%""",0.02
"""75%""",0.03
"""max""",0.61


In [47]:
df.head()

DATE,TIME_M,EX,BID,BIDSIZ,ASK,ASKSIZ,QU_COND,QU_SEQNUM,NATBBO_IND,QU_CANCEL,QU_SOURCE,SYM_ROOT,SYM_SUFFIX
str,time,str,f64,i64,f64,i64,str,i64,i64,str,str,str,str
"""2023-05-11""",09:30:00.000536359,"""Q""",173.77,9,173.92,1,"""R""",801372,2,null,"""N""","""AAPL""",null
"""2023-05-11""",09:30:00.000633151,"""Z""",173.57,1,174.71,1,"""R""",801385,0,null,"""N""","""AAPL""",null
"""2023-05-11""",09:30:00.001056573,"""Z""",173.57,1,174.71,1,"""R""",801418,0,null,"""N""","""AAPL""",null
"""2023-05-11""",09:30:00.002307163,"""K""",173.37,1,174.0,31,"""R""",801487,0,null,"""N""","""AAPL""",null
"""2023-05-11""",09:30:00.002449230,"""U""",173.78,1,173.92,1,"""R""",801505,2,null,"""N""","""AAPL""",null


In [ ]:
df.with_columns(
    pl.when(
                (pl.col("TIME_M") < pl.time(*(10, 35, 865))) |
                (pl.col("TIME_M") < pl.time(*(12, 35, 865))) 
            )
    .then(0)
    )

DATE,TIME_M,EX,BID,BIDSIZ,ASK,ASKSIZ,QU_COND,QU_SEQNUM,NATBBO_IND,QU_CANCEL,QU_SOURCE,SYM_ROOT,SYM_SUFFIX,literal
str,time,str,f64,i64,f64,i64,str,i64,i64,str,str,str,str,i32
"""2023-05-11""",09:30:00.000536359,"""Q""",173.77,9,173.92,1,"""R""",801372,2,null,"""N""","""AAPL""",null,null
"""2023-05-11""",09:30:00.000633151,"""Z""",173.57,1,174.71,1,"""R""",801385,0,null,"""N""","""AAPL""",null,null
"""2023-05-11""",09:30:00.001056573,"""Z""",173.57,1,174.71,1,"""R""",801418,0,null,"""N""","""AAPL""",null,null
"""2023-05-11""",09:30:00.002307163,"""K""",173.37,1,174.0,31,"""R""",801487,0,null,"""N""","""AAPL""",null,null
"""2023-05-11""",09:30:00.002449230,"""U""",173.78,1,173.92,1,"""R""",801505,2,null,"""N""","""AAPL""",null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2023-05-11""",15:59:59.992303426,"""V""",172.04,4,207.18,1,"""R""",88797645,0,null,"""N""","""TSLA""",null,null
"""2023-05-11""",15:59:59.992917650,"""Q""",172.04,163,172.08,80,"""R""",88797672,4,null,"""N""","""TSLA""",null,null
"""2023-05-11""",15:59:59.993692002,"""V""",172.04,4,172.08,2,"""R""",88797698,0,null,"""N""","""TSLA""",null,null


In [ ]:
df.with_columns(
    pl.when(
        (pl.col("TIME_M") < pl.time(10, 35)) |
        (pl.col("TIME_M") > pl.time(12, 35))
    )
    .then(pl.lit(0))
    .alias("FILTER_FLAG")
)

DATE,TIME_M,EX,BID,BIDSIZ,ASK,ASKSIZ,QU_COND,QU_SEQNUM,NATBBO_IND,QU_CANCEL,QU_SOURCE,SYM_ROOT,SYM_SUFFIX,FILTER_FLAG
str,time,str,f64,i64,f64,i64,str,i64,i64,str,str,str,str,i32
"""2023-05-11""",09:30:00.000536359,"""Q""",173.77,9,173.92,1,"""R""",801372,2,null,"""N""","""AAPL""",null,0
"""2023-05-11""",09:30:00.000633151,"""Z""",173.57,1,174.71,1,"""R""",801385,0,null,"""N""","""AAPL""",null,0
"""2023-05-11""",09:30:00.001056573,"""Z""",173.57,1,174.71,1,"""R""",801418,0,null,"""N""","""AAPL""",null,0
"""2023-05-11""",09:30:00.002307163,"""K""",173.37,1,174.0,31,"""R""",801487,0,null,"""N""","""AAPL""",null,0
"""2023-05-11""",09:30:00.002449230,"""U""",173.78,1,173.92,1,"""R""",801505,2,null,"""N""","""AAPL""",null,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2023-05-11""",15:59:59.992303426,"""V""",172.04,4,207.18,1,"""R""",88797645,0,null,"""N""","""TSLA""",null,null
"""2023-05-11""",15:59:59.992917650,"""Q""",172.04,163,172.08,80,"""R""",88797672,4,null,"""N""","""TSLA""",null,null
"""2023-05-11""",15:59:59.993692002,"""V""",172.04,4,172.08,2,"""R""",88797698,0,null,"""N""","""TSLA""",null,null


In [58]:
df.with_columns(
            pl.when(
                (pl.col("TIME_M") < pl.time(*(10, 35, 865))) | 
                (pl.col("TIME_M") > pl.time(*(10, 36, 865)))
            )
            .then(0)
            .otherwise(pl.col("Signal"))
            .alias("Signal")
        )

ColumnNotFoundError: Signal

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'sink' <---
DF ["DATE", "TIME_M", "EX", "BID", ...]; PROJECT */14 COLUMNS

In [ ]:
df_tmp.head(100).to_pandas().plot.line(x="QU_SEQNUM", y="OBI")

ArrowInvalid: Value 37800048115245 has non-zero nanoseconds

In [8]:
del filtered_df
del average_spread

# Data prep
1. filter trading hours
2. Nasdaq quotes
3. take twosided quotes only

In [9]:
# filter trading hours
# df = df.with_columns(
#     pl.col("TIME_M").str.to_time(format="%H:%M:%S%.f").alias("TIME_M")
# )
df = df.filter(pl.col("SYM_ROOT") == "AAPL")
df = df.filter(
    pl.col("TIME_M").is_between(pl.time(10, 30), pl.time(12, 0))
)
# Filter for Nasdaq exchange
df = df.filter(pl.col("EX") == "Z")
# Filter for rows where QU_COND is 'R'
df = df.filter(pl.col("QU_COND") == "R")
df.shape

(346766, 14)

In [40]:
df.head()

DATE,TIME_M,EX,BID,BIDSIZ,ASK,ASKSIZ,QU_COND,QU_SEQNUM,NATBBO_IND,QU_CANCEL,QU_SOURCE,SYM_ROOT,SYM_SUFFIX
str,time,str,f64,i64,f64,i64,str,i64,i64,str,str,str,str
"""2023-05-11""",10:30:00.002054882,"""Z""",172.67,5,172.68,2,"""R""",18779360,0,null,"""N""","""AAPL""",null
"""2023-05-11""",10:30:00.002273381,"""Z""",172.67,5,172.68,2,"""R""",18779388,0,null,"""N""","""AAPL""",null
"""2023-05-11""",10:30:00.002358341,"""Z""",172.67,3,172.68,2,"""R""",18779394,2,null,"""N""","""AAPL""",null
"""2023-05-11""",10:30:00.002371580,"""Z""",172.67,2,172.68,2,"""R""",18779395,0,null,"""N""","""AAPL""",null
"""2023-05-11""",10:30:00.002397118,"""Z""",172.67,1,172.68,2,"""R""",18779398,0,null,"""N""","""AAPL""",null


In [10]:
df.describe()

statistic,DATE,TIME_M,EX,BID,BIDSIZ,ASK,ASKSIZ,QU_COND,QU_SEQNUM,NATBBO_IND,QU_CANCEL,QU_SOURCE,SYM_ROOT,SYM_SUFFIX
str,str,str,str,f64,f64,f64,f64,str,f64,f64,str,str,str,str
"""count""","""346766""","""346766""","""346766""",346766.0,346766.0,346766.0,346766.0,"""346766""",346766.0,346766.0,"""0""","""346766""","""346766""","""0"""
"""null_count""","""0""","""0""","""0""",0.0,0.0,0.0,0.0,"""0""",0.0,0.0,"""346766""","""0""","""0""","""346766"""
"""mean""",null,"""11:10:23.756954""",null,173.18669,4.189508,173.200275,4.225492,null,2.7777e7,0.176661,null,null,null,null
"""std""",null,null,null,0.379069,3.312605,0.379182,2.971931,null,5.3706e6,0.576804,null,null,null,null
"""min""","""2023-05-11""","""10:30:00.002054""","""Z""",172.3,1.0,172.31,1.0,"""R""",1.877936e7,0.0,null,"""N""","""AAPL""",null
"""25%""",null,"""10:47:12.383551""",null,172.97,2.0,172.99,2.0,null,2.315817e7,0.0,null,null,null,null
"""50%""",null,"""11:07:04.075073""",null,173.3,4.0,173.32,4.0,null,2.7565596e7,0.0,null,null,null,null
"""75%""",null,"""11:33:49.644025""",null,173.48,5.0,173.49,6.0,null,3.2540505e7,0.0,null,null,null,null
"""max""","""2023-05-11""","""11:59:59.882232""","""Z""",173.87,64.0,173.89,53.0,"""R""",3.7019465e7,4.0,null,"""N""","""AAPL""",null


In [11]:
df['NATBBO_IND'].value_counts()

NATBBO_IND,count
i64,u32
0,316595
4,459
2,29712


### National Best Bid-Offer (NBBO) - Legend 
‘0’ = No National BBO change - Current quote does not affect the BBO. No National appendage is required.

‘1’ = No National BBO Can be Calculated- The National BBO cannot be calculated therefore vendors should show National BBO fields as blank. No Appendage is required.

‘2’ = Short Form National BBO Appendage Attached – A new National BBO was generated as a result of the UTP participant’s quote update and the new information is contained in the short form appendage (NBBO FILE)

‘3’ = Long Format of National BBO Appendage - A new National BBO is generated and the new BBO information is contained in the Long National BBO appendage (NBBO FILE)

‘4’ = Quote Contains all NASD BBO Information - Current quote is itself the new NASD BBO. No NASD appendage is required.

---
In the case above, I will track all bid offer and size regardless of NATBBO indicator

In [12]:
df['QU_COND'].value_counts()

QU_COND,count
str,u32
"""R""",346766


In [13]:
df.glimpse

<bound method DataFrame.glimpse of shape: (346_766, 14)
┌────────────┬──────────────────┬─────┬────────┬───┬───────────┬───────────┬──────────┬────────────┐
│ DATE       ┆ TIME_M           ┆ EX  ┆ BID    ┆ … ┆ QU_CANCEL ┆ QU_SOURCE ┆ SYM_ROOT ┆ SYM_SUFFIX │
│ ---        ┆ ---              ┆ --- ┆ ---    ┆   ┆ ---       ┆ ---       ┆ ---      ┆ ---        │
│ str        ┆ time             ┆ str ┆ f64    ┆   ┆ str       ┆ str       ┆ str      ┆ str        │
╞════════════╪══════════════════╪═════╪════════╪═══╪═══════════╪═══════════╪══════════╪════════════╡
│ 2023-05-11 ┆ 10:30:00.0020548 ┆ Z   ┆ 172.67 ┆ … ┆ null      ┆ N         ┆ AAPL     ┆ null       │
│            ┆ 82               ┆     ┆        ┆   ┆           ┆           ┆          ┆            │
│ 2023-05-11 ┆ 10:30:00.0022733 ┆ Z   ┆ 172.67 ┆ … ┆ null      ┆ N         ┆ AAPL     ┆ null       │
│            ┆ 81               ┆     ┆        ┆   ┆           ┆           ┆          ┆            │
│ 2023-05-11 ┆ 10:30:00.0023583 ┆ Z

In [14]:
df.columns

['DATE',
 'TIME_M',
 'EX',
 'BID',
 'BIDSIZ',
 'ASK',
 'ASKSIZ',
 'QU_COND',
 'QU_SEQNUM',
 'NATBBO_IND',
 'QU_CANCEL',
 'QU_SOURCE',
 'SYM_ROOT',
 'SYM_SUFFIX']

# Try OBI strategy

In [15]:
class OBIVWAPStrategy:
    def __init__(self, vwap_window: int, obi_threshold: float, initial_cash: float = 100_000):
        self.vwap_window = vwap_window
        self.obi_threshold = obi_threshold
        self.cash = initial_cash
        self.position = 0
        self.account_balance = []

    def calculate_vwap(self, df: pl.DataFrame) -> pl.DataFrame:
        # Calculate MID_PRICE
        df = df.with_columns(
            ((pl.col("BID") + pl.col("ASK")) / 2).alias("MID_PRICE")
        )
        # Calculate Volume
        df = df.with_columns(
            (pl.col("BIDSIZ") + pl.col("ASKSIZ")).alias("Volume")
        )
        # Calculate VWAP using rolling window
        df = df.with_columns(
            (
                (pl.col("MID_PRICE") * pl.col("Volume"))
                .rolling_sum(window_size=self.vwap_window)
                / pl.col("Volume").rolling_sum(window_size=self.vwap_window)
            ).alias("VWAP")
        )
        return df

    def calculate_obi(self, df: pl.DataFrame) -> pl.DataFrame:
        # Calculate Order Book Imbalance (OBI)
        df = df.with_columns(
            (
                (pl.col("BIDSIZ") - pl.col("ASKSIZ"))
                / (pl.col("BIDSIZ") + pl.col("ASKSIZ"))
            ).alias("OBI")
        )
        return df

    def generate_signals(self, df: pl.DataFrame) -> pl.DataFrame:
        # Calculate VWAP and OBI
        df = self.calculate_vwap(df)
        df = self.calculate_obi(df)

        # Generate signals based on OBI threshold
        df = df.with_columns(
            pl.when(pl.col("OBI") > self.obi_threshold)
            .then(1)  # Buy signal
            .when(pl.col("OBI") < -self.obi_threshold)
            .then(-1)  # Sell signal
            .otherwise(0)  # No signal
            .alias("Signal")
        )
        return df

    def backtest(self, df: pl.DataFrame) -> pl.DataFrame:
        # Initialize account balance tracking
        account_balance = []

        # Iterate over rows to simulate trading
        for row in df.iter_rows(named=True):
            if row["Signal"] == 1 and self.cash >= row["ASK"] * 100 and self.position <= 1:
                # Buy 100 shares
                self.position = 100
                self.cash -= row["ASK"] * 100
            elif row["Signal"] == -1 and self.position > -1:
                # Sell 100 shares
                self.position = -100
                self.cash += row["BID"] * 100
            elif row["Signal"] == 0 and self.position != 0:
                # Close position
                if self.position > 0:
                    self.cash += row["BID"] * self.position
                else:
                    self.cash -= row["ASK"] * abs(self.position)
                self.position = 0

            # Record account balance
            account_balance.append(self.cash + self.position * (row["ASK"] if self.position > 0 else row["BID"]))

        # Add account balance to the DataFrame
        df = df.with_columns(pl.Series("Account_Balance", account_balance))
        return df

In [16]:
strategy = OBIVWAPStrategy(vwap_window=50, obi_threshold=0.2)
signal_data = strategy.generate_signals(df)
backtest_data = strategy.backtest(signal_data)

print(backtest_data.head())


shape: (5, 20)
┌────────────┬────────────────────┬─────┬────────┬───┬──────┬───────────┬────────┬─────────────────┐
│ DATE       ┆ TIME_M             ┆ EX  ┆ BID    ┆ … ┆ VWAP ┆ OBI       ┆ Signal ┆ Account_Balance │
│ ---        ┆ ---                ┆ --- ┆ ---    ┆   ┆ ---  ┆ ---       ┆ ---    ┆ ---             │
│ str        ┆ time               ┆ str ┆ f64    ┆   ┆ f64  ┆ f64       ┆ i32    ┆ f64             │
╞════════════╪════════════════════╪═════╪════════╪═══╪══════╪═══════════╪════════╪═════════════════╡
│ 2023-05-11 ┆ 10:30:00.002054882 ┆ Z   ┆ 172.67 ┆ … ┆ null ┆ 0.428571  ┆ 1      ┆ 100000.0        │
│ 2023-05-11 ┆ 10:30:00.002273381 ┆ Z   ┆ 172.67 ┆ … ┆ null ┆ 0.428571  ┆ 1      ┆ 100000.0        │
│ 2023-05-11 ┆ 10:30:00.002358341 ┆ Z   ┆ 172.67 ┆ … ┆ null ┆ 0.2       ┆ 0      ┆ 99999.0         │
│ 2023-05-11 ┆ 10:30:00.002371580 ┆ Z   ┆ 172.67 ┆ … ┆ null ┆ 0.0       ┆ 0      ┆ 99999.0         │
│ 2023-05-11 ┆ 10:30:00.002397118 ┆ Z   ┆ 172.67 ┆ … ┆ null ┆ -0.333333 ┆ -1

In [17]:
backtest_data["Account_Balance"].describe()

statistic,value
str,f64
"""count""",346766.0
"""null_count""",0.0
"""mean""",597322.933682
"""std""",380869.031193
"""min""",4275.0
"""25%""",214422.0
"""50%""",609627.0
"""75%""",937708.0
"""max""",1.384521e6


In [18]:
backtest_data.columns

['DATE',
 'TIME_M',
 'EX',
 'BID',
 'BIDSIZ',
 'ASK',
 'ASKSIZ',
 'QU_COND',
 'QU_SEQNUM',
 'NATBBO_IND',
 'QU_CANCEL',
 'QU_SOURCE',
 'SYM_ROOT',
 'SYM_SUFFIX',
 'MID_PRICE',
 'Volume',
 'VWAP',
 'OBI',
 'Signal',
 'Account_Balance']

In [19]:
# df_tmp = backtest_data.filter(pl.col("VWAP").is_not_null())

In [20]:
backtest_data['OBI'].describe()

statistic,value
str,f64
"""count""",346766.0
"""null_count""",0.0
"""mean""",-0.0069
"""std""",0.407902
"""min""",-0.962963
"""25%""",-0.333333
"""50%""",0.0
"""75%""",0.333333
"""max""",0.967213


In [21]:
df_tmp = backtest_data.filter(pl.col("VWAP").is_not_null())
df_tmp

DATE,TIME_M,EX,BID,BIDSIZ,ASK,ASKSIZ,QU_COND,QU_SEQNUM,NATBBO_IND,QU_CANCEL,QU_SOURCE,SYM_ROOT,SYM_SUFFIX,MID_PRICE,Volume,VWAP,OBI,Signal,Account_Balance
str,time,str,f64,i64,f64,i64,str,i64,i64,str,str,str,str,f64,i64,f64,f64,i32,f64
"""2023-05-11""",10:30:00.048115245,"""Z""",172.66,1,172.67,5,"""R""",18780753,0,null,"""N""","""AAPL""",null,172.665,6,172.673403,-0.666667,-1,82720.0
"""2023-05-11""",10:30:00.048210867,"""Z""",172.66,1,172.67,6,"""R""",18780758,0,null,"""N""","""AAPL""",null,172.665,7,172.673194,-0.714286,-1,82720.0
"""2023-05-11""",10:30:00.048252344,"""Z""",172.65,4,172.67,6,"""R""",18780760,0,null,"""N""","""AAPL""",null,172.66,10,172.672766,-0.2,0,82719.0
"""2023-05-11""",10:30:00.050295683,"""Z""",172.65,5,172.67,6,"""R""",18780809,0,null,"""N""","""AAPL""",null,172.66,11,172.672326,-0.090909,0,82719.0
"""2023-05-11""",10:30:00.050368773,"""Z""",172.66,1,172.67,6,"""R""",18780813,0,null,"""N""","""AAPL""",null,172.665,7,172.672147,-0.714286,-1,82719.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2023-05-11""",11:59:59.657033606,"""Z""",173.58,6,173.59,1,"""R""",37018932,0,null,"""N""","""AAPL""",null,173.585,7,173.585,0.714286,1,227425.0
"""2023-05-11""",11:59:59.668570673,"""Z""",173.58,7,173.59,1,"""R""",37019118,0,null,"""N""","""AAPL""",null,173.585,8,173.585,0.75,1,227425.0
"""2023-05-11""",11:59:59.670847290,"""Z""",173.58,8,173.59,1,"""R""",37019136,0,null,"""N""","""AAPL""",null,173.585,9,173.585,0.777778,1,227425.0


In [39]:
df_tmp[['QU_SEQNUM', 'OBI']].head(1000).plot.line(x="QU_SEQNUM", y="OBI")

alt.Chart(...)

In [37]:
df_tmp[['QU_SEQNUM', 'MID_PRICE']].head(1000).plot.line(x="QU_SEQNUM", y="MID_PRICE")

alt.Chart(...)

In [36]:
df_tmp[['QU_SEQNUM', 'VWAP']].head(1000).plot.line(x="QU_SEQNUM", y="VWAP")

alt.Chart(...)

In [26]:
df_tmp.head(100).to_pandas().plot.line(x="QU_SEQNUM", y="OBI")

ArrowInvalid: Value 37800048115245 has non-zero nanoseconds

In [23]:
df_tmp.head(2000).plot.line(x="TIME_M", y="MID_PRICE")

ValueError: Unexpected DtypeKind: Time

alt.Chart(...)

In [24]:
df_tmp.head(2000).plot.line(x="TIME_M", y="OBI")

ValueError: Unexpected DtypeKind: Time

alt.Chart(...)

In [25]:
df_tmp.head(2000).

SyntaxError: invalid syntax (3424061299.py, line 1)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Convert to pandas
backtest_data = backtest_data.with_columns(
    (pl.col("DATE").cast(pl.Utf8) + " " + pl.col("TIME_M").cast(pl.Utf8))
    .str.strptime(pl.Datetime("ns"), format="%Y-%m-%d %H:%M:%S%.f")
    .alias("TIME_M")
)

pdf = backtest_data.to_pandas()


# Plot
for ticker in set(pdf['SYM_ROOT']):
    ticker_df = pdf[pdf['SYM_ROOT'] == ticker]
    plt.figure(figsize=(12, 6))
    plt.plot(ticker_df["TIME_M"], ticker_df["Account_Balance"])
    plt.xlabel("TIME_M")
    plt.ylabel("Account_Balance")
    plt.title(f"Account Balance for {ticker}")
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
    plt.xticks(rotation=45)
    plt.show()

InvalidOperationError: conversion from `str` to `datetime[ns]` failed in column 'DATE' for 346766 out of 346766 values: ["2023-05-11 2023-05-11 10:30:00.000000000", "2023-05-11 2023-05-11 10:30:00.000000000", … "2023-05-11 2023-05-11 11:59:59.000000000"]

You might want to try:
- setting `strict=False` to set values that cannot be converted to `null`
- using `str.strptime`, `str.to_date`, or `str.to_datetime` and providing a format string